In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [ ]:
# necessary imports
import os
import gc
import cv2
import numpy as np
import pandas as pd
from PIL import Image

import timm

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
DATA_DIR = "../input/ranzcr-clip-catheter-line-classification"

plt.style.use('bmh')
plt.rcParams['figure.figsize'] = [20, 13]
SEED = 421

In [ ]:
IMAGE_SIZE = 600
BATCH_SIZE = 2
MODEL_NAME = "efficientnet_b7"
MODEL_PATH_effnetb7 = "../input/ranzcr-final-timm-efficientnetb7/timm-effnet-b7-res600-final.pt"
TTA = True

In [ ]:
input_df = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
print (input_df.info())

### Utility function

In [ ]:
def plot_input_images(imgs: torch.Tensor, title_string: str, nrow: int = 4) -> None:
    image_grid = make_grid(imgs, nrow=nrow, padding=10, pad_value=1)
    
    # transform from CHW -> HWC
    plt.imshow(image_grid.permute(1, 2, 0), cmap=plt.cm.bone)
    plt.title(title_string)

### Test datasets for both models

In [ ]:
CLASSES = [col for col in input_df.columns if col not in ['StudyInstanceUID']]


class RanczrDataset(Dataset):
    def __init__(self, df, data_dir, transform=None, is_test=False, is_rgb=False):
        super().__init__()
        self.df = df
        self.data_dir = data_dir
        self.transform = transform
        self.is_test = is_test
        self.is_rgb = is_rgb
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        data = {}
        
        img_name = self.df['StudyInstanceUID'][index]
        
        if not self.is_test:
            targets = self.df.loc[index, CLASSES].values.astype(np.uint8)
        
            targets = torch.from_numpy(targets)
            
            data['targets'] = targets
            
        img_path = os.path.join(self.data_dir, img_name+".jpg")
        
        
        if self.is_rgb:
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        else:
            image = Image.open(img_path)
            
        if self.transform:
            image = self.transform(image)
        
        data['image'] = image
        
        return data

### Transforms

In [ ]:
test_effnet_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

# test_resnet200d_transforms = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])


### Visualising test dataset samples for both models

In [ ]:
# creating the dataset object
test_dataset = RanczrDataset(input_df, os.path.join(DATA_DIR, "test"), test_effnet_transforms, is_test=True)

# creating the test dataloader
test_loader1 = DataLoader(dataset=test_dataset, batch_size=8, num_workers=0, shuffle=False)

batch = next(iter(test_loader1))

plot_input_images(batch['image'], title_string='batch images', nrow=8)

del batch
del test_dataset
del test_loader1
gc.collect()

In [ ]:
# # creating the dataset object
# test_dataset = RanczrDataset(input_df, os.path.join(DATA_DIR, "test"), test_resnet200d_transforms, is_test=True, is_rgb=True)

# # creating the test dataloader
# test_loader2 = DataLoader(dataset=test_dataset, batch_size=8, num_workers=0, shuffle=False)

# batch = next(iter(test_loader2))

# plot_input_images(batch['image'], title_string='batch images', nrow=8)

# del batch
# del test_dataset
# del test_loader2
# gc.collect()

### Model

In [ ]:
class Effnetb7(nn.Module):
    def __init__(self, model_name='efficientnet_b7'):
        super(Effnetb7, self).__init__()
        
        self.effnet = timm.create_model(model_name, pretrained=False, in_chans=1).as_sequential()[:-2]
        
        self.dropout = nn.Dropout(p=0.5)
        self.dense = nn.Linear(2560, len(CLASSES))
        
        
    def forward(self, images):
        pooled_features= self.effnet(images)
        
        outputs = self.dense(self.dropout(pooled_features))
        
        return outputs
    
# class RANZCRResNet200D(nn.Module):
#     def __init__(self, model_name='resnet200d', out_dim=11, pretrained=False):
#         super().__init__()
#         self.model = timm.create_model(model_name, pretrained=False)
#         n_features = self.model.fc.in_features
#         self.model.global_pool = nn.Identity()
#         self.model.fc = nn.Identity()
#         self.pooling = nn.AdaptiveAvgPool2d(1)
#         self.fc = nn.Linear(n_features, out_dim)

#     def forward(self, x):
#         bs = x.size(0)
#         features = self.model(x)
#         pooled_features = self.pooling(features).view(bs, -1)
#         output = self.fc(pooled_features)
#         return output

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

effnetb7 = Effnetb7()
effnetb7.load_state_dict(torch.load(MODEL_PATH_effnetb7, map_location='cuda:0'))
effnetb7 = effnetb7.to(device)
# resnet200d = RANZCRResNet200D()
# resnet200d.load_state_dict(torch.load(MODEL_PATH_resnet200d, map_location='cuda:0'))

### Utilities

In [ ]:
# Adapted from https://www.kaggle.com/underwearfitting/resnet200d-public-benchmark-2xtta-lb0-965#Utils

def inference_func(model, test_loader, device):
    model.eval()
    bar = tqdm(test_loader)
    LOGITS = []
    PREDS = []
    
    with torch.no_grad():
        for batch_idx, data in enumerate(bar):
            x = data['image'].to(device)
            logits = model(x)
            LOGITS.append(logits.cpu())
            PREDS += [logits.sigmoid().detach().cpu()]
        PREDS = torch.cat(PREDS).cpu().numpy()
        LOGITS = torch.cat(LOGITS).cpu().numpy()
    return PREDS

def tta_inference_func(model, test_loader):
    model.eval()
    bar = tqdm(test_loader)
    PREDS = []
    LOGITS = []

    with torch.no_grad():
        for batch_idx, data in enumerate(bar):
            x = data['image'].to(device)
            x = torch.stack([x,x.flip(2),x.flip(3)],0) # TTA: vflip and hflip
            x = x.view(-1, 1, IMAGE_SIZE, IMAGE_SIZE)
            logits = model(x)
            logits = logits.view(BATCH_SIZE, 3, -1).mean(1)
            PREDS += [logits.sigmoid().detach().cpu()]
            LOGITS.append(logits.cpu())
        PREDS = torch.cat(PREDS).cpu().numpy()
        
    return PREDS

### Predictions for Efficientnet-b7

In [ ]:
# creating the dataset object
test_dataset = RanczrDataset(input_df, os.path.join(DATA_DIR, "test"), test_effnet_transforms, is_test=True)

# creating the test dataloader
test_loader_effnet = DataLoader(dataset=test_dataset, batch_size=2, num_workers=0, shuffle=False)

test_preds = None
if TTA:
    print ('Using TTA')
    test_preds = tta_inference_func(effnetb7, test_loader_effnet)
else:
    test_preds = inference_func(effnetb7, test_loader_effnet)

In [ ]:
test_preds

In [ ]:
submission = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

for i in range(test_preds.shape[1]):
    submission.loc[:, CLASSES[i]] = test_preds[:, i]

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
# # creating the dataset object
# test_dataset = RanczrDataset(input_df, os.path.join(DATA_DIR, "test"), test_resnet200d_transforms, is_test=True, is_rgb=True)

# # creating the test dataloader
# test_loader_resnet200d = DataLoader(dataset=test_dataset, batch_size=2, num_workers=0, shuffle=False)

# predictions_resnet200d = inference_func(resnet200d.to(device), test_loader_resnet200d, device)

In [ ]:
# predictions = (2 * predictions_resnet200d + predictions_effnetb7) / 3.0